In [1]:
from skyfield.api import PlanetaryConstants, load
from skyfield.timelib import Time

In [2]:

ts = load.timescale()
tn=ts.now()
t = ts.utc(2019, 12, 20, 11, 5)

In [3]:
eph = load('de421.bsp')
earth, moon = eph['earth'], eph['moon']

pc = PlanetaryConstants()
pc.read_text(load('moon_080317.tf'))
pc.read_text(load('pck00008.tpc'))
pc.read_binary(load('moon_pa_de421_1900-2050.bpc'))

In [4]:
frame = pc.build_frame_named('MOON_ME_DE421')
aristarchus = moon + pc.build_latlon_degrees(frame, 26.3, -46.8)

# From earth to moon

In [5]:
apparent = earth.at(t).observe(aristarchus).apparent()  # use time 't' for 20/12/1019 and 'tn' for time now
ra, dec, distance = apparent.radec(epoch='date')

In [6]:
# time 20/12/2019 (E2M)
print(ra)
print(dec)
d=str(distance)
print(float(d[:-2])*149597870.7, 'KM')

13h 03m 22.96s
-00deg 55' 27.3"
369880.7353057499 KM


In [7]:
# time now (E2M)
print(ra)
print(dec)
d=str(distance)
print(float(d[:-2])*149597870.7, 'KM')

13h 03m 22.96s
-00deg 55' 27.3"
369880.7353057499 KM


# From moon to earth


In [8]:
apparent = aristarchus.at(tn).observe(earth).apparent()   # use time 't' for 20/12/1019 and 'tn' for time now
ra, dec, distance = apparent.radec(epoch='date')

In [9]:
# time 20/12/2019 (M2E)
print(ra)
print(dec)
d=str(distance)
print(float(d[:-2])*149597870.7, 'KM')

17h 29m 22.87s
-23deg 41' 33.3"
395400.6360684629 KM


In [10]:
# time now (M2E)
print(ra)
print(dec)
d=str(distance)
print(float(d[:-2])*149597870.7, 'KM')

17h 29m 22.87s
-23deg 41' 33.3"
395400.6360684629 KM


In [11]:
d


'0.00264309 au'

In [12]:
0.00268879*149597870.7

402237.25875945296

## altitude and azimuth measured against the astronaut’s horizon

In [13]:
# from moon to earth
alt, az, distance = apparent.altaz()
print(alt, 'degrees above the horizon')
print(az, 'degrees around the horizon from north')

40deg 42' 20.9" degrees above the horizon
116deg 05' 36.9" degrees around the horizon from north


# Sub-Solar points on the moon


In [14]:
sun = eph['Sun']

p = moon.at(t).observe(sun).apparent()
lat, lon, distance = p.frame_latlon(frame)
lon_degrees = (lon.degrees + 180.0) % 360.0 - 180.0
print('Sub-solar latitude: {:.1f} degrees'.format(lat.degrees))
print('Sub-solar longitude: {:.1f} degrees'.format(lon_degrees))

Sub-solar latitude: 0.3 degrees
Sub-solar longitude: -104.9 degrees


# Lunar Libration

In [15]:
p = (earth - moon).at(t)
lat, lon, distance = p.frame_latlon(frame)
lon_degrees = (lon.degrees + 180.0) % 360.0 - 180.0
print('Libration in latitude: {:.3f}'.format(lat.degrees))
print('Libration in longitude: {:.3f}'.format(lon_degrees))

Libration in latitude: -6.749
Libration in longitude: 1.520


# Computing a raw rotation matrix¶


In [16]:
R = frame.rotation_at(t)
print(R.shape)

(3, 3)


In [17]:
t = ts.utc(2019, 12, 20, 11, range(5, 15))
R = frame.rotation_at(t)
print(t.shape)
print(R.shape)

(10,)
(3, 3, 10)
